<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/dev/Progetto_Bozza_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%rm -r sample_data
!wget --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
!mkdir -p ffhq/train/real
!mkdir -p ffhq/train/synthesized
!mkdir -p ffhq/test/real
!mkdir -p ffhq/test/synthesized
%cd /content/0_real/
!mv $(ls | shuf -n 2500) ../ffhq/train/real/
!mv * ../ffhq/test/real/
%cd ..
!rm -d 0_real

!pip install tensorflow_io

In [ ]:
# synthesized examples are missing, this block of code is purely for testing grammar and syntax for the rest of the script
# !cp ffhq/train/real/* ffhq/train/synthesized/
# !cp ffhq/test/real/* ffhq/test/synthesized/

In [ ]:
!%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf

In [ ]:
batch_size = 32
interpolation = 'bilinear'
train_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5, # Validation is 1/6 of the total, so 1/5 of (total - test)
    subset='training',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)
val_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5,
    subset='validation',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)

In [ ]:
k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.dtypes.float32
    )
print(k)

In [ ]:
# Preprocessing
import tensorflow_io as tfio

d = 3
def randomGaussianBlur(x):
    k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.float32
    )
    return tfio.experimental.filter.gaussian(x, d, k)

def randomRot90(x):
    k = tf.random.uniform(
        shape=[],
        minval=0,
        maxval=4,
        dtype=tf.dtypes.int32,
        seed=None,
        name=None
    )
    return tf.image.rot90(x, k)

def preprocess(x):
    y = tf.keras.layers.CenterCrop(224,224)(x)
    y = tf.keras.layers.Rescaling(1./127.5, offset=-1)(x)
    return y

def augment(x):
    y = randomGaussianBlur(x)
    y = randomRot90(x)
    return y

val_dataset = val_dataset.map(lambda x, y: (preprocess(x), y))
train_dataset = train_dataset.map(lambda x, y: (augment(preprocess(x)), y))

In [ ]:
# Reference: https://www.tensorflow.org/tutorials/load_data/images#configure_the_dataset_for_performance
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(256,256,3),
    pooling=None, # hyperparameter?
)
#train_dataset = tf.keras.applications.xception.preprocess_input(train_dataset)
#val_dataset = tf.keras.applications.xception.preprocess_input(val_dataset)

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
)
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction=tf.keras.losses.Reduction.AUTO,
)
metric = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='ROC',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False # ??
)

train_after_layer = 100
for layer in base_model.layers[:train_after_layer]:
    layer.trainable = False

model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalMaxPooling2D())
model.add(tf.keras.layers.Dense(1, activation='softmax'))

model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metric
)
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

batch_size=1000
epochs = 2
for i in range(51):
    model.fit(
        train_dataset,
        batch_size=batch_size,
        epochs=(1+i)*epochs,
        initial_epoch=i*epochs,
        verbose="auto",
        #validation_data=val_dataset,
        #validation_freq=1,
        shuffle=True,
    )
    loss, AUC = model.evaluate(val_dataset)
    model.save_weights('drive/MyDrive/pesi_ffhq/%d_%.3f_%.3f.h5' %((i+1)*epochs, loss, AUC))